In [3]:
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.vizier import Vizier
from filter_functions import filter_on_archival, filter_on_chandra, filter_on_gaia

In [4]:
detections = pd.read_csv('detections_w20.txt', sep=' ', header=0)

for i, detection in detections.loc[0:10].iterrows():
    print(i)
    filter_on_gaia(detection)

0


gaierror: [Errno 11002] getaddrinfo failed